# Notebook 02: Data Augmentation (Negative Sampling)

**Mục tiêu:** Sinh dữ liệu huấn luyện cho model Machine Learning (Pairwise Ranking).

Để train model, ta cần tạo ra các cặp dữ liệu `(Query, Candidate, Label)`:
* **Query:** Thông tin trích xuất từ BibTeX (đã clean).
* **Candidate:** Thông tin từ Ground Truth (Metadata).
* **Label:** 1 (Match) hoặc 0 (Non-match).

**Chiến lược sinh mẫu sai (Negative Sampling):**
1.  **Local Negatives (Hard):** Chọn metadata sai nằm *trong cùng một bài báo* (`paper_id` giống nhau). Ví dụ: BibTeX ref #1 ghép với GT ref #2 của cùng bài báo đó. Đây là mẫu khó vì cùng chủ đề.
2.  **Global Negatives (Easy):** Chọn metadata sai từ *bất kỳ bài báo nào khác*. Giúp model học phân biệt sự khác biệt rõ rệt (khác năm, khác tác giả...).



In [109]:
import pandas as pd
import numpy as np
import random
import os
import sys

# Setup path để import src module
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import từ src.ml module
from src.ml import (
    load_pickle,
    save_pickle,
    augment_dataset,
    build_candidate_pools,
    split_by_partition
)

# --- CẤU HÌNH ---
INPUT_FILE = '../../dataset_final/clean_data/cleaned_data.pkl'
OUTPUT_FILE = '../../dataset_final/clean_data/train_augmented.pkl'

# Tỷ lệ Negative/Positive
NUM_NEGATIVES = 4  
NUM_HARD_NEGATIVES = 2 

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print("✅ Import modules thành công từ src.ml!")

✅ Import modules thành công từ src.ml!



## 1. Load dữ liệu sạch (Cleaned Data)
Ta load file `.pkl` được tạo ra từ Notebook 01 để tận dụng các trường `clean_title`, `clean_authors`, `clean_id`.


In [110]:
# Load dữ liệu sạch sử dụng hàm từ module
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"❌ Không tìm thấy file: {INPUT_FILE}. Hãy chạy Notebook 01 trước.")

df_all = load_pickle(INPUT_FILE)

# Lấy cả train + validation để augment
df_train_src = df_all[df_all['partition'].isin(['train', 'validation'])].copy()

print(f"Tổng số mẫu gốc (Train + Validation): {len(df_train_src)}")
print("Các cột có sẵn:", df_train_src.columns.tolist())

Tổng số mẫu gốc (Train + Validation): 13455
Các cột có sẵn: ['partition', 'source_file', 'key', 'paper_id', 'bib_content', 'clean_title', 'clean_authors', 'clean_id', 'clean_year', 'parse_method', 'gt_id', 'gt_title', 'gt_authors', 'gt_year']



## 2. Xây dựng Candidate Pools
Ta cần tạo kho chứa các ứng viên (Ground Truth) để lấy mẫu.


In [111]:
# Sử dụng hàm build_candidate_pools từ module
global_candidates, local_pool = build_candidate_pools(df_train_src)

print(f"Global Pool Size: {len(global_candidates)}")
print(f"Number of Papers (Local Pools): {len(local_pool)}")

Global Pool Size: 13455
Number of Papers (Local Pools): 610


## 3. Thực hiện Negative Sampling
Quy trình cho mỗi dòng BibTeX (Query):
1.  Tạo 1 cặp **Positive** (Chính nó).
2.  Tạo k cặp **Hard Negative** (Lấy từ `local_pool` của cùng `paper_id`).
3.  Tạo m cặp **Easy Negative** (Lấy ngẫu nhiên từ `global_candidates` khác `paper_id`).


In [112]:
# Sử dụng hàm augment_dataset từ module
# Hàm này đã bao gồm toàn bộ logic sinh Positive + Negative samples

df_augmented = augment_dataset(
    df_source=df_train_src,
    num_negatives=NUM_NEGATIVES,
    num_hard_negatives=NUM_HARD_NEGATIVES,
    random_seed=RANDOM_SEED,
    verbose=True
)

📊 Global Pool Size: 13455
📊 Number of Papers: 610


🚀 Generating samples: 100%|██████████| 13455/13455 [00:00<00:00, 21628.23it/s]



✅ Augmentation completed!
   - Original samples: 13455
   - Augmented samples: 67275
   - Label distribution:
label
0    53820
1    13455
Name: count, dtype: int64


## 4. Lưu kết quả
Lưu thành DataFrame để dùng cho bước Feature Engineering.


In [113]:
# Lưu file sử dụng module
print("\n--- KẾT QUẢ DATA AUGMENTATION ---")
print(f"Số lượng mẫu ban đầu: {len(df_train_src)}")
print(f"Số lượng mẫu sau khi sinh: {len(df_augmented)}")
print(f"Tỷ lệ Positive/Negative:\n{df_augmented['label'].value_counts()}")

# Lưu file PKL
df_augmented.to_pickle(OUTPUT_FILE)

print(f"\n✅ Đã lưu file dataset huấn luyện tại: {os.path.abspath(OUTPUT_FILE)}")
print("👉 BƯỚC TIẾP THEO: Chạy '03_feature_engineering.ipynb'")


--- KẾT QUẢ DATA AUGMENTATION ---
Số lượng mẫu ban đầu: 13455
Số lượng mẫu sau khi sinh: 67275
Tỷ lệ Positive/Negative:
label
0    53820
1    13455
Name: count, dtype: int64

✅ Đã lưu file dataset huấn luyện tại: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\dataset_final\clean_data\train_augmented.pkl
👉 BƯỚC TIẾP THEO: Chạy '03_feature_engineering.ipynb'


## 5. Kiểm tra mẫu dữ liệu (Sanity Check)
Kiểm tra xem mẫu Hard Negative trông như thế nào.


In [114]:

# %%
# Lấy 1 mẫu Positive
print("--- MẪU POSITIVE ---")
pos_sample = df_augmented[df_augmented['label'] == 1].iloc[0]
print(f"Bib Title:  {pos_sample['bib_title']}")
print(f"Cand Title: {pos_sample['cand_title']}")
print(f"Match ID:   {pos_sample['bib_id']} == {pos_sample['cand_id']}")

print("\n--- MẪU NEGATIVE (Check xem Title có khác nhau không) ---")
neg_sample = df_augmented[df_augmented['label'] == 0].iloc[0]
print(f"Bib Title:  {neg_sample['bib_title']}")
print(f"Cand Title: {neg_sample['cand_title']}")
print(f"Label:      {neg_sample['label']}")

--- MẪU POSITIVE ---
Bib Title:  Ibrnet: Learning multi-view image-based rendering
Cand Title: IBRNet: Learning Multi-View Image-Based Rendering
Match ID:    == 2102-13090

--- MẪU NEGATIVE (Check xem Title có khác nhau không) ---
Bib Title:  Graph States as a Resource for Quantum Metrology
Cand Title: Emergent Bartering Behaviour in Multi-Agent Reinforcement Learning
Label:      0


In [115]:
# lưu json để check

import json
with open('sample_positive.json', 'w', encoding='utf-8') as f:
    json.dump(pos_sample.to_dict(), f, ensure_ascii=False, indent=4)